In [1]:
import torch
from model import Model 
import configparser

In [2]:
def read_config_file(file_path):
    # 创建 ConfigParser 对象
    config = configparser.ConfigParser()
    # 读取配置文件
    config.read(file_path)
    return config

config_file_path = 'hyper-parameters.ini'
config = read_config_file(config_file_path)

batch_size = config.getint('Hyperparameters', 'batch_size')
context_length = config.getint('Hyperparameters', 'batch_size')
d_model = config.getint('Hyperparameters', 'd_model')
num_blocks = config.getint('Hyperparameters', 'num_blocks')
num_heads = config.getint('Hyperparameters', 'num_heads')
learning_rate = config.getfloat('Hyperparameters', 'learning_rate')
dropout = config.getfloat('Hyperparameters', 'dropout')
max_iters = config.getint('Hyperparameters', 'max_iters')
eval_interval = config.getint('Hyperparameters', 'eval_interval')
eval_iters = config.getint('Hyperparameters', 'eval_iters')
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # Use GPU if it's available.
TORCH_SEED = 1337
torch.manual_seed(TORCH_SEED)

In [3]:
with open('data/scifi.txt', 'r') as f:
    text = f.read()

In [4]:
vocab = sorted(list(set(text)))
vocab_size = max_token_value = len(vocab)
char2idx = {c:i for i,c in enumerate(vocab)}
idx2char = {i:c for i,c in enumerate(vocab)}
encode = lambda x: [char2idx[c] for c in x]
decode = lambda idxs: ''.join([idx2char[i] for i in idxs])
tokenized_text = torch.tensor(encode(text), dtype=torch.long)


In [7]:
model = Model().to(device)

In [9]:
model.load_state_dict(torch.load('model/model-ckpt.pt'))
model.eval()

Model(
  (token_embedding_lookup_table): Embedding(100001, 64)
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (multi_head_attention_layer): MultiHeadAttention(
        (heads): ModuleList(
          (0-7): 8 x Attention(
            (key_layer): Linear(in_features=64, out_features=8, bias=False)
            (query_layer): Linear(in_features=64, out_features=8, bias=False)
            (value_layer): Linear(in_features=64, out_features=8, bias=False)
            (dropout_layer): Dropout(p=0.1, inplace=False)
          )
        )
        (projection_layer): Linear(in_features=64, out_features=64, bias=True)
        (dropout_layer): Dropout(p=0.1, inplace=False)
      )
      (feed_forward_layer): FeedForward(
        (ffn): Sequential(
          (0): Linear(in_features=64, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=64, bias=True)
          (3): Dropout(p=0.1, inplace=False)
        )
      )
      (layer_no

In [11]:
start = '奥特曼出生在一个小村庄'
start_ids = encode(start)


In [12]:
x = (torch.tensor(torch.tensor(start_ids,dtype=torch.long,device=device)[None,...]))


/tmp/ipykernel_776/2442113015.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = (torch.tensor(torch.tensor(start_ids,dtype=torch.long,device=device)[None,...]))


In [13]:
y = model.generate(x)
print('-----------------')
print(decode(y[0].tolist()))
print('-----------------')

-----------------
奥特曼出生在一个小村庄，紧们不必要醒，她问，无荷纳也不是安全失罗？一次在一条小树，种月闪齐恰队属，似乎吃发出了一颗绍禁适栗有漆。”在那种两十二件，而且，没见回答：“你们也没有发生。”
但是大再度地提她的光线散已在一个维底球
-----------------
